In [31]:
import requests
import time #this will prevent us from DDOSing their systems
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

In [5]:
coordinates = pd.DataFrame([[38.917221, 38.910525, 38.766335, 38.910004, 38.864772, 38.9497058], 
               [-77.0415997, -77.02365, -77.147626, -77.020183, -76.973936, -77.0118382]])

In [60]:
def uberProducts(auth_token, s_latitude, s_longitude, e_latitude, e_longitude): 
    with requests.session() as c:
        url = 'https://api.uber.com/v1.2/estimates/price'
        query = {'start_latitude':  s_latitude, 'start_longitude': s_longitude, 'end_latitude': e_latitude, 'end_longitude' : e_longitude}
        headers = {'Accept-Language': 'en_US', 'Content-Type': 'application/json', 'Authorization': 'Token ' + auth_token }
        time.sleep(.5)
        r = requests.get(url, params=query, headers=headers)
        print r.status_code, r.reason, ' - ', r.url
        return r

def import_response_to_pandas(response):
    """
    For Uber API v1.2.
    This function converts a successful API response and returns a pandas dataframe.
    If the response was not successful, it returns an empty list.
    """
    response_as_dict = json.loads(response.content)
    if 'prices' in response_as_dict:
        return pd.read_json(json.dumps(response_as_dict['prices']))
    else:
        return []


groupPool = range(0, 100, 10)

for i in coordinates:
    response = uberProducts('P3yvujYYMqULTwg5X9euvOYvR1sNQ0w9as-hQ2Ub', 38.9048728,-77.036217, coordinates[i][0], coordinates[i][1] )
    response.content
    time.sleep(.25)
    df_add = import_response_to_pandas(response)
    if i == 0:
        Data = df_add
    else:
        Data = Data.append(df_add, ignore_index=True)

200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-77.0415997&start_latitude=38.9048728&end_latitude=38.917221
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-77.02365&start_latitude=38.9048728&end_latitude=38.910525
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-77.147626&start_latitude=38.9048728&end_latitude=38.766335
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-77.020183&start_latitude=38.9048728&end_latitude=38.910004
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-76.973936&start_latitude=38.9048728&end_latitude=38.864772
200 OK  -  https://api.uber.com/v1.2/estimates/price?start_longitude=-77.036217&end_longitude=-77.0118382&start_latitude=38.9048728&end_latitude=38.9497058


In [66]:
df_type = Data[ (Data['display_name'] == 'POOL') | (Data['display_name'] == 'uberX') ]
display_name = df_type.groupby('localized_display_name').sum()
display_name

,distance,duration,high_estimate,low_estimate
localized_display_name,,,,
POOL,30.46,4680,81.0,57.0
uberX,30.46,4680,102.0,76.0
